In [50]:
import sys
import pickle
import numpy as np

from train import load_pickle
from utils import Vocab
from adem_data_loader import get_adem_loader
from adem_configs import get_adem_config

from adem_model import ADEM

In [2]:
sys.argv = [
    'train.py',
    
    # Mode
    '--mode', 'train',
    
    # Generation
    '--temperature', '1.0',
    '--beam_size', '5',
    
    # Model    
    '--model', 'ADEM',
    '--context_size', '400',
    '--embedding_size', '300',
    '--encoder_hidden_size', '400',
    '--bidirectional', 'True',
    '--decoder_hidden_size', '800',
    
    '--data', 'smart_ko_adem',
]

In [3]:
adem_config = get_adem_config(sys.argv)

In [4]:
print('Loading Vocabulary...')
vocab = Vocab()
vocab.load(adem_config.word2id_path, adem_config.id2word_path)
print(f'Vocabulary size: {vocab.vocab_size}')

Loading Vocabulary...
Vocabulary size: 63625


In [5]:
adem_config.vocab_size = vocab.vocab_size

In [6]:
adem_data_loader = get_adem_loader(
    sentences=load_pickle(adem_config.sentences_path),
    conversation_length=load_pickle(adem_config.conversation_length_path),
    sentence_length=load_pickle(adem_config.sentence_length_path),
    score=load_pickle(adem_config.score_path),
    vocab=vocab,
    batch_size=64,
    shuffle=False)

In [7]:
adem = ADEM(adem_config)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [8]:
pretrained_path = 'pretrained_vhred.pkl'
adem.load_pretrained_model(pretrained_path)

encoder.embedding.weight
encoder.rnn.weight_ih_l0
encoder.rnn.weight_hh_l0
encoder.rnn.bias_ih_l0
encoder.rnn.bias_hh_l0
encoder.rnn.weight_ih_l0_reverse
encoder.rnn.weight_hh_l0_reverse
encoder.rnn.bias_ih_l0_reverse
encoder.rnn.bias_hh_l0_reverse
context_encoder.rnn.weight_ih_l0
context_encoder.rnn.weight_hh_l0
context_encoder.rnn.bias_ih_l0
context_encoder.rnn.bias_hh_l0


In [15]:
adem

ADEM(
  (encoder): EncoderRNN(
    (embedding): Embedding(63625, 300, padding_idx=0)
    (rnn): GRU(300, 400, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (context_encoder): ContextRNN(
    (rnn): GRU(800, 400, batch_first=True, dropout=0.2)
  )
)

In [46]:
adem_emb = list(adem.encoder.embedding.parameters())[0].detach().numpy()
adem_emb

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.3339272 ,  0.98614734, -1.881724  , ..., -0.27759275,
        -0.25167027,  0.9045324 ],
       [-0.90530425,  0.04456139, -2.0799513 , ...,  0.558683  ,
         0.26754048, -0.37137714],
       ...,
       [ 1.4056088 ,  0.05423704, -1.088891  , ..., -1.0354873 ,
         0.32125464,  1.4435543 ],
       [ 0.26614946,  1.3424605 ,  0.76443064, ...,  1.2876545 ,
         0.89401686,  0.38838202],
       [ 0.1791303 ,  0.13732897,  1.7306836 , ...,  0.38411716,
         1.2699164 ,  1.4800925 ]], dtype=float32)

In [47]:
pretrained_weights = torch.load('pretrained_vhred.pkl')['encoder.embedding.weight'].cpu().detach().numpy()
pretrained_weights

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.3339272 ,  0.98614734, -1.881724  , ..., -0.27759275,
        -0.25167027,  0.9045324 ],
       [-0.90530425,  0.04456139, -2.0799513 , ...,  0.558683  ,
         0.26754048, -0.37137714],
       ...,
       [ 1.4056088 ,  0.05423704, -1.088891  , ..., -1.0354873 ,
         0.32125464,  1.4435543 ],
       [ 0.26614946,  1.3424605 ,  0.76443064, ...,  1.2876545 ,
         0.89401686,  0.38838202],
       [ 0.1791303 ,  0.13732897,  1.7306836 , ...,  0.38411716,
         1.2699164 ,  1.4800925 ]], dtype=float32)

In [52]:
np.array_equal(adem_emb, pretrained_weights)

True